<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-불러오기" data-toc-modified-id="데이터-불러오기-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>데이터 불러오기</a></span></li><li><span><a href="#전처리" data-toc-modified-id="전처리-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>전처리</a></span><ul class="toc-item"><li><span><a href="#명사추출" data-toc-modified-id="명사추출-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>명사추출</a></span></li><li><span><a href="#word2vec(유의어-처리)" data-toc-modified-id="word2vec(유의어-처리)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>word2vec(유의어 처리)</a></span></li><li><span><a href="#bow(불용어처리)" data-toc-modified-id="bow(불용어처리)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>bow(불용어처리)</a></span></li></ul></li><li><span><a href="#새로운-query를-포함한-문서-추출" data-toc-modified-id="새로운-query를-포함한-문서-추출-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>새로운 query를 포함한 문서 추출</a></span><ul class="toc-item"><li><span><a href="#tf-idf" data-toc-modified-id="tf-idf-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>tf-idf</a></span><ul class="toc-item"><li><span><a href="#클러스터링" data-toc-modified-id="클러스터링-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>클러스터링</a></span><ul class="toc-item"><li><span><a href="#kmeans" data-toc-modified-id="kmeans-3.1.1.1"><span class="toc-item-num">3.1.1.1&nbsp;&nbsp;</span>kmeans</a></span></li><li><span><a href="#lsa" data-toc-modified-id="lsa-3.1.1.2"><span class="toc-item-num">3.1.1.2&nbsp;&nbsp;</span>lsa</a></span></li></ul></li></ul></li><li><span><a href="#doc2vec" data-toc-modified-id="doc2vec-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>doc2vec</a></span><ul class="toc-item"><li><span><a href="#클러스터링" data-toc-modified-id="클러스터링-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>클러스터링</a></span></li></ul></li></ul></li><li><span><a href="#kmeans" data-toc-modified-id="kmeans-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>kmeans</a></span></li></ul></div>

In [ ]:
!pip install tensorflow
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pickle
import datetime
import re

import tensorflow as tf

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

## 데이터 불러오기

In [2]:
def get_dataframe(data_name_with_route):
    ## load news data
    with open(data_name_with_route, 'rb') as file:
        data_list = []
        while True:
            try:
                data = pickle.load(file)
            except EOFError:
                break
            data_list.append(data)
    ## construct lists for dataframe
    title = []
    content = []
    date = []
    for news in data_list[0]['return_object']['documents']: 
        title.append(news['title'])
        content.append(news['content'])
        date.append(news['published_at'][:10]) #### 시간 조정이 필요하면 바꾸기
    ## make lists as dataframe
    news_data = pd.DataFrame([])
    news_data['date'] = date
    news_data['title'] = title
    news_data['content'] = content
    return news_data

In [3]:
data_son = get_dataframe('rawdata_손흥민.pickle')
data_kbl = get_dataframe('rawdata_프로농구.pickle')

In [4]:
data_son['date_tmp'] = data_son['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').toordinal())
data_kbl['date_tmp'] = data_kbl['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').toordinal())

In [5]:
data_son['label'] = 0
data_kbl['label'] = 1

In [6]:
data = pd.concat([data_son,data_kbl])
data.reset_index(inplace = True)
data.drop(['index'],axis = 1,inplace = True)

## 전처리

In [7]:
reg_reporter = re.compile('[가-힣]+\s[가-힣]*기자') # 기자
reg_email = re.compile('[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$') # 이메일
reg_eng = re.compile('[a-z]+') # 소문자 알파벳, 이메일 제거용, 대문자는 남겨둔다
reg_chi = re.compile("[\u4e00-\u9fff]+") # 한자
reg_sc = re.compile("·|ㆍ|ㆍ|…|◆+|◇+|▶+|●+|▲+|“|”|‘|’|\"|\'|\(|\)") # 특수문자
reg_date = re.compile('\d*일|\d*월|\d*년|\d*시|\d*분|\(현지시간\)|\(현지시각\)|\d+') #날짜,시간,숫자

In [8]:
def preProcessing(doc):
    tmp = re.sub(reg_reporter, '', doc)
    tmp = re.sub(reg_email, '', tmp)
    tmp = re.sub(reg_eng, '', tmp)
    tmp = re.sub(reg_chi, '', tmp)
    tmp = re.sub(reg_sc, ' ', tmp)
    tmp = re.sub(reg_date, '', tmp)
    return tmp

In [9]:
data['content_re'] = data['content'].apply(preProcessing)

### 명사추출

In [19]:
!pip install customized_konlpy

In [20]:
from ckonlpy.tag import Twitter
twitter = Twitter()

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [21]:
## 사전 계속 추가
add_dict = ['마우리시오', 
            '포체티노', 
            '아시안게임', 
            '그리고',]
twitter.add_dictionary(add_dict, 'Noun')

In [22]:
##불용어는 나중에 계속 추가!
##text로 정리하기
stop_basic = '''
아
휴
'''
stop_words = stop_basic.split()

In [23]:
tokenized_doc = data['content_re'].apply(lambda x: twitter.nouns(x))
tokenized_doc2 = tokenized_doc.apply(lambda x: [item.lower() for item in x if item not in stop_words])
display(tokenized_doc.head())
display(tokenized_doc2.head())

0    [내, 챔스리그, 강, 티, 강, 확정, 손, 마, 우리, 오, 포체티노, 사진, ...
1    [미국, 이, 아아, 최고, 스포츠스타, 손흥민, 토트넘, 집중, 조명, 은, 한국...
2    [크루이프, 후계, 자, 아약스, 크리스티아누, 호날두, 유벤투스, 를, 네덜란드,...
3    [네덜란드, 아약스, 크리스티아누, 호날두, 해, 이탈리아, 유벤투스, 유럽, 축구...
4    [지난, 경기, 광주, 경찰, 기숙, 학원, 오전, 이, 걸그룹, 트와이스, 예스,...
Name: content_re, dtype: object

0    [내, 챔스리그, 강, 티, 강, 확정, 손, 마, 우리, 오, 포체티노, 사진, ...
1    [미국, 이, 아아, 최고, 스포츠스타, 손흥민, 토트넘, 집중, 조명, 은, 한국...
2    [크루이프, 후계, 자, 아약스, 크리스티아누, 호날두, 유벤투스, 를, 네덜란드,...
3    [네덜란드, 아약스, 크리스티아누, 호날두, 해, 이탈리아, 유벤투스, 유럽, 축구...
4    [지난, 경기, 광주, 경찰, 기숙, 학원, 오전, 이, 걸그룹, 트와이스, 예스,...
Name: content_re, dtype: object

### word2vec(유의어 처리)

In [12]:
!pip install gensim

  Stored in directory: C:\Users\USER\AppData\Local\pip\Cache\wheels\2a\12\79\a26b1d566ac0edbcc806689fdf4f813ff4aaa5b5cf6e37406b
  Stored in directory: C:\Users\USER\AppData\Local\pip\Cache\wheels\83\58\f0\a1f22e381370565b08cca8216eb82c0e716f70de28720e26ed
  Stored in directory: C:\Users\USER\AppData\Local\pip\Cache\wheels\81\75\d6\e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built gensim smart-open bz2file


In [24]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=tokenized_doc2, size=300, window=5, min_count=5, workers=4, sg=1)
tmp = model.wv.most_similar("손흥민")

### bow(불용어처리)

In [25]:
tmp_list = []
for doc in tokenized_doc2:
    tmp_list.extend(doc)

In [26]:
word_count = pd.Series(tmp_list).value_counts()

In [27]:
word_count_idx = list(word_count.index)

In [28]:
stop_words = word_count_idx[:10] + word_count_idx[-10:] + ['내']

In [29]:
data['content_token'] = tokenized_doc2.apply(lambda x: [item.lower() for item in x if item not in stop_words])

## 새로운 query를 포함한 문서 추출

In [30]:
query = '경기'

In [36]:
selected_news = []
for idx in range(tokenized_doc.shape[0]):
    if query in tokenized_doc.iloc[idx] :
        selected_news.append(idx)

In [46]:
len(selected_news)

7407

In [48]:
query_doc = tokenized_doc.iloc[selected_news]

In [50]:
len(query_doc)

7407

### tf-idf

In [51]:
tmp = query_doc.apply(lambda x : ' '.join(x))

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

obj_tfidf = TfidfVectorizer()
x = obj_tfidf.fit_transform(tmp)

In [53]:
sim_mat = 1 - np.round(pairwise_distances(x, metric="cosine"),3)
np.fill_diagonal(sim_mat,-1) # 같은 문서는 유사도 -1로 처리

In [54]:
sim_mat

array([[-1.   ,  0.22 ,  0.115, ...,  0.019,  0.036,  0.06 ],
       [ 0.22 , -1.   ,  0.064, ...,  0.007,  0.028,  0.029],
       [ 0.115,  0.064, -1.   , ...,  0.005,  0.013,  0.004],
       ...,
       [ 0.019,  0.007,  0.005, ..., -1.   ,  0.372,  0.408],
       [ 0.036,  0.028,  0.013, ...,  0.372, -1.   ,  0.301],
       [ 0.06 ,  0.029,  0.004, ...,  0.408,  0.301, -1.   ]])

#### 클러스터링

kmeans, 코사인 유사도, lsa

##### kmeans

In [176]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 2,random_state=0).fit(x)

In [189]:
kmeans_label = kmeans.labels_
label = data['label'][selected_news]

In [191]:
np.sum((label==kmeans_label)*1)/len(kmeans_label)

0.9823140272715

##### lsa

In [60]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(x)
len(svd_model.components_)

2

In [61]:
np.shape(svd_model.components_)

(2, 24159)

In [62]:
terms = obj_tfidf.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('손흥민', 0.33699), ('경기', 0.21996), ('한국', 0.20551), ('감독', 0.18736), ('선수', 0.17705)]
Topic 2: [('체육관', 0.229), ('우리은행', 0.22765), ('쿼터', 0.2164), ('프로농구', 0.20393), ('모비스', 0.18167)]


In [65]:
topic0 = svd_model.components_[0]
topic1= svd_model.components_[1]

In [70]:
x_array = x.toarray()

In [71]:
x_array.shape

(7407, 24159)

In [72]:
from scipy import spatial

In [122]:
topic_0_sim = []
topic_1_sim = []

for doc in x_array:
    topic_0_sim.append(1-spatial.distance.cosine(topic0,doc))
for doc in x_array:
    topic_1_sim.append(1-spatial.distance.cosine(topic1,doc))


lsa 수치가 너무 작아 0 과 1 를 비교해서 새로 labeling을 했습니다.

In [ ]:
topic_0_sim = pd.Series(topic_0_sim)
topic_1_sim = pd.Series(topic_1_sim)

df_topic_sim = pd.concat([topic_0_sim,topic_1_sim],axis = 1)
df_topic_sim['topic_label'] = df_topic_sim.apply(lambda row : 0 if row[0]>=row[1] else 1,axis = 1)
#topic_0_idx = topic_0_sim[topic_0_sim>0.4].index.tolist()
#topic_1_idx = topic_1_sim[topic_1_sim>0.4].index.tolist()

#label_topic = label.copy(deep = True)
#label_topic = label_topic.apply(lambda x : 0)

#label_topic.iloc[topic_0_idx] = 0
#label_topic.iloc[topic_1_idx] = 1

In [156]:
np.mean((label.values==df_topic_sim['topic_label'].values)*1)

0.7669771837451059

In [159]:
df_topic_sim['topic_label'].value_counts()

0    5590
1    1817
Name: topic_label, dtype: int64

In [161]:
label.value_counts()

0    3866
1    3541
Name: label, dtype: int64

### doc2vec

#### 클러스터링

## kmeans

In [195]:
kmeans_label==0

array([ True,  True,  True, ..., False, False, False])